# lxml.etree的使用
以下是用`lxml.etree`处理XML文件的使用指导，主要简短描述了ElementTree API的主要功能。
首先，导入模块：

In [1]:
from lxml import etree
from lxml import objectify

### 元素类
ElementTree API的主要数据承载对象时元素。大部分XML树可以通过元素的方式访问。

In [2]:
# 创建一个ElementTree对象，根节点为root，对象名为root
root = etree.Element("root")

XML的标签名称可以通过`tag` 属性访问：

In [3]:
print(root.tag)

root


元素是XML树结构的主要组成部分，在树中加入子节点可以使用`append`

In [4]:
root.append(etree.Element("child1"))

当然，我们也有更简单的方法添加子节点： 

In [5]:
child2 = etree.SubElement(root, "child2")
child3 = etree.SubElement(root, "child3")

这样就形成了一个真实的XML文档，你可以查看他的结构：

In [6]:
print(etree.tostring(root, pretty_print=True))

b'<root>\n  <child1/>\n  <child2/>\n  <child3/>\n</root>\n'


### 元素以列表形式存在
想要简单直接的访问这些子节点，元素可以利用类似于普通Pyhton列表的一些方法。

In [7]:
child = root[0]
print(child.tag)

child1


In [8]:
print(len(root))

3


In [9]:
root.index(root[1])  # 需要导入objectify模块才可以使用！

1

In [10]:
children = list(root)
# 同样可以进行循环的操作
for child in root:
    print(child.tag)

child1
child2
child3


In [11]:
root.insert(0, etree.Element("child0"))
# 同样可以进行切片的操作
start= root[:1]
end = root[-1:]
print(start[0].tag)
print(end[0].tag)

child0
child3


查看是否是节点元素类型

In [12]:
print(etree.iselement(root))

True


查看根节点是否还有子节点 

In [13]:
if len(root):
    print("The root element has children")

The root element has children


### 元素节点的覆盖移动
这一点和列表不同，列表的赋值是拷贝，而ElementTree树的操作是直接覆盖移动

In [14]:
for child in root:
    print(child.tag)
root[0] = root[-1]
print("*"*30)
for child in root:
    print(child.tag)

child0
child1
child2
child3
******************************
child3
child1
child2


如果你想将节点元素拷贝到`lxml.etree`中的新位置，就要使用Python的copy模块中的深拷贝`deep copy`来完成。

In [15]:
from copy import deepcopy
# 创建一个名为elemet的对象，根节点名为neu
element = etree.Element("neu")
# 通过深拷贝（新内存地址）向对象添加子节点
element.append(deepcopy(root[1]))
print(element[0].tag)
# 显然，并没有移走原来的root[1]节点
print([c.tag for c in root])

child1
['child3', 'child1', 'child2']


通过兄弟节点也可以互相访问

In [16]:
root[0] is root[1].getprevious()
root[1] is root[0].getnext()

True

### 节点元素以字典的方式携带属性
XML元素支持属性，通过字典的方式携带元素属性

In [17]:
root = etree.Element("root", interesting="totally")
etree.tostring(root)

b'<root interesting="totally"/>'

元素的属性是无序的键值对，所以一个方便的方式来处理他们就是类似Python的字典处理方式 

In [18]:
# 使用 根节点.get("属性名") 来获取元素的属性信息
print(root.get("interesting"))
print(root.get("hello"))
# 可以使用set来设置属性
root.set("hello", "Hengheng")
print(root.get("hello"))
for name, value in sorted(root.items()):
    print('%s = %s' % (name, value))

totally
None
Hengheng
hello = Hengheng
interesting = totally


如果你想要的真的类字典操作，可以进行如下处理：

In [19]:
attributes = root.attrib
print(attributes["interesting"])
print(attributes.get("no-such-attribute"))
attributes["hello"] = "tag"
print(attributes.get("hello"))

totally
None
tag


# 从字符串或者文件解析

`lxml.tree`支持以各种方式解析XML文件，可以解析字符串、文件、URLs或者其他类文件对象。主要的解析函数是`fromstring()`和`parse()`。

### `fromstring()函数`

`fromstring()`可以轻松的解析一个字符串：

In [20]:
# 字符串
some_xml_data = "<root>data</root>"
# 使用fromstring来解析
root = etree.fromstring(some_xml_data)
print(root.tag)
etree.tostring(root)

root


b'<root>data</root>'

### `XML()`函数

`xml()`函数和`fromstring()`函数功能类似，但是一般把要解析的XML文本直接写入在其中：

In [21]:
root = etree.XML("<root>data</root>")
print(root.tag)
etree.tostring(root)

root


b'<root>data</root>'

### `parse()`函数

`parse()`函数用来处理文件和类文件的数据。注意，`parse()`函数返回以一个元素树对象，而不是一个元素。与字符串处理方法不同的是，处理完文件之后`parse()`返回的是一个完整的文件，而字符串只是用来处理一个XML段落。

### 解析器对象

默认来说，`lxml.etree`使用标准解析来处理，如果你想要自定义解析器，你就需要在定义解析器时指定参数。

In [22]:
# etree.XMLParser??

In [23]:
# 自定义一个解析器对象，选择我需要的功能
parser = etree.XMLParser(remove_blank_text = True)
# 注意：最里面的叶子节点的元素不会被去空格，因为最内层叶子中被认为是数据值。
root = etree.XML("<root> <a/>   <b>  </b>    </root>", parser)
etree.tostring(root)

b'<root><a/><b>  </b></root>'

#### 分步解析

`lxml.etree`提供了两种一步步解析的方法。一种针对类文件，最合适是用再`urllib`，比如：

In [24]:
class DataSource:
    data = [ b"<roo", b"t><", b"a/", b"><", b"/root>" ]
    def read(self, requested_size):
        try:
            return self.data.pop(0)
        except IndexError:
            return b''
        
tree = etree.parse(DataSource())
etree.tostring(tree)

b'<root><a/></root>'

第二种方法是通过`feed`一步步传入数据，最后用`close()`结尾。

In [25]:
parser = etree.XMLParser()
parser.feed("<roo")
parser.feed("t><")
parser.feed("a/")
parser.feed("><")
parser.feed("/root>")
root = parser.close()
etree.tostring(root)

b'<root><a/></root>'

#### 事件驱动型解析

有时候你所需要解析的内容只是整个文件的一小部分，如果整个文件解析成元素树，那么就是深处的叶子节点，与其说将整个树都解析到内存中，不如使用`lxml.etree`提供的`iterparse()`方法。下面是一个简单的例子：

In [33]:
from io import BytesIO
some_file_like = BytesIO(b"<root><a>data</a></root>")
for event, element in etree.iterparse(some_file_like):
    print("%s, %4s, %s" % (event, element.tag, element.text))

end,    a, data
end, root, None


就默认情况下，`iterparse()` 对元素解析仅仅生成一个事件，不过你可以通过`events`关键词来修改它。

In [34]:
some_file_like = BytesIO(b"<root><a>data</a></root>")
for event, element in etree.iterparse(some_file_like, events=("start", "end")):
    print("%5s, %4s, %s" % (event, element.tag, element.text))
# 注意，这种情况下只有当end结束才意味着整个XMl解析完成

start, root, None
start,    a, data
  end,    a, data
  end, root, None


同样的，他允许你用`.clear()` 方法，也允许修改树中某个节点的值。所以在当树过大而你又想要节省内存空间的时候，可以使用`.clear()`来清空一部分树。参数`keep_tail = True`和`.clear()`方法保证了尾部的文件内容不会被清除。非常不建议在没有解析完全部文本的时候修改任何的内容。

In [39]:
some_file_like = BytesIO(b"<root><a><b>data</b></a><a><b/></a></root>")
for event, element in etree.iterparse(some_file_like):
    if element.tag == 'b':
        print(element.text)
    elif element.tag == 'a':
        print("** cleaning up the subtree")
        element.clear()

data
** cleaning up the subtree
None
** cleaning up the subtree


 `iterparse()`的一个重要用途是用来解析大型XML文件。这些文件都是上面有一个根节点，而下面是成百上千的子节点。

In [43]:
xml_file = BytesIO(b'''\
    <root>
        <a><b>ABC</b><c>abc</c></a>
        <a><b>MORE DATA</b><c>more data</c></a>
        <a><b>XYZ</b><c>xyz</c></a>
     </root>''')

for _, element in etree.iterparse(xml_file, tag='a'):
    # element[1].text表示对其中取值，findtext(b)表示寻找<b>标签中的值
    print('%s -- %s' % (element.findtext('b'), element[1].text))
    element.clear()

ABC -- abc
MORE DATA -- more data
XYZ -- xyz


 **待更新**

# Xpath语法

`lxml.etree`支持简单的路径语法来*查找*、*查找所有*、*查找文本*。

### 选取节点
XPath使用路径表达式在XML文档中选取节点。节点是通过沿着路径或者step来选取的。

**以下是常用的路径表达式：**

`nodename`：选取此节点的所有子节点

`/`：从根节点选取（绝对选取）

`//`：从匹配选择的当前节点选择文档中的节点，而不考虑他们的位置（相对选取）

`.`：选取当前节点

`..`：选取当前节点的子节点

`@`：选取属性

### 谓语（Predicates） 
谓语用来查找某个特定的节点或者包含某个指定的值得节点，谓语被镶嵌在方括号中。

`/bookstore/book[1]`：选取属于 bookstore 子元素的第一个 book 元素。

`/bookstore/book[last()]`	选取属于 bookstore 子元素的最后一个 book 元素。

`/bookstore/book[last()-1]`	选取属于 bookstore 子元素的倒数第二个 book 元素。

`/bookstore/book[position()<3]`	选取最前面的两个属于 bookstore 元素的子元素的 book 元素。

`//title[@lang]`	选取所有拥有名为 lang 的属性的 title 元素。

`//title[@lang='eng']`	选取所有 title 元素，且这些元素拥有值为 eng 的 lang 属性。

`/bookstore/book[price>35.00]`	选取 bookstore 元素的所有 book 元素，且其中的 price 元素的值须大于 35.00。

`/bookstore/book[price>35.00]/title`	选取 bookstore 元素中的 book 元素的所有 title 元素，且其中的 price 元素的值须大于 35.00。

### 选取未知节点

XPath通配符可用来选取未知的XMl元素

`*`：匹配任何元素节点。

`@*`：匹配任何属性节点。

`node()`：匹配任何类型的节点。

### 选取若干路径

通过在路径表达式中使用"|"运算符，您可以选取若若干个路径。

**暂略**

## lxml.objectify模块的使用
在使用`objectify`的时候，既会用到`lxml.etree`模块也会用到`lxml.object`模块。

In [ ]:
from lxml import etree
from lxml import objectify

### 通过对象属性访问元素
`objectify`API和`ElementTree`API不同，一旦使用这个API就不应该与别的元素处理混合使用。
这个API的主要思想是将XML元素的访问方法隐藏于普通元素对象的访问方法中。通过元素属性访问，将返回带有相应标签名称的子元素序列。


In [ ]:
root = objectify.Element("root")
b = objectify.SubElement(root, "b")
# 根节点下子节点b的第一个元素的标签值
print(root.b[0].tag)
# 节点b的第一个元素的下标
# root.index(root.b[0])
# 更简洁的写法，可以省略下标0来访问第一个孩子节点
root.index(root.b)

*暂略*

#  用lxml解析XML和HTML

对于XMl和HTMl的解析，lxml提供了一个简单有效的API。
简单的步骤如下：

In [ ]:
from lxml import etree

In [ ]:
# 下面的例子是使用IO模块进行文件的导入的
from io import StringIO, BytesIO

### 解析器

In [ ]:
xml = '<a xmlns="test"><b xmlns="test"/></a>'
root = etree.fromstring(xml)
etree.tostring(root)

从一个文件读取对象，你可以使用`prase()`函数，会返回一个元素树对象

In [ ]:
tree = etree.parse(StringIO(xml))
etree.tostring(tree.getroot())

**我们现在从本地读取文件：**

In [ ]:
tree = etree.parse("C:\\Users\\Administrator\\Desktop\\CFturbo机器学习课题\\临时泵模型\\混流1.cft-batch")

### 解析的选项

解析时我们接受一个关键字参数作为设置选项，通过这个我们可以轻松的清理命名空间

In [ ]:
parser = etree.XMLParser(ns_clean=True)
tree = etree.parse(StringIO(xml), parser)
etree.tostring(tree.getroot())

In [ ]:
tree = etree.parse("C:\\Users\\Administrator\\Desktop\\CFturbo机器学习课题\\临时泵模型\\混流1.cft-batch")
root = tree.getroot()

 **待更新**